<a href="https://colab.research.google.com/github/gunelaliyevaa/wildfire-detection-using-satellite-imagery/blob/main/scripts/image_extraction_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fire Event Capture and Visualization using Sentinel-2 Imagery

This code authenticates and initializes Google Earth Engine (GEE) with a specified project, then processes wildfire locations to generate visualization URLs from Sentinel-2 satellite imagery.

The code performs the following steps:

1. **Data Loading**: Reads fire data from a CSV file with columns `LATITUDE`, `LONGITUDE`, and `ACQ_DATE`.
2. **Iterates Through Fire Events**: Treats each row in the CSV as a wildfire event, creating a ~1 km rectangular area of interest around each location.
3. **Image Processing**:
   - Fetches Sentinel-2 images for each area within a 1-day range of the event date.
   - Applies cloud and cirrus masking using Sentinel-2’s `QA60` band, filtering images with less than 20% cloud cover.
   - Selects RGB bands (Red, Green, Blue) and creates a median composite of available images for visualization.
4. **Output Generation**: Generates and stores a list of dictionaries, each containing:
   - `latitude` and `longitude`: Coordinates of the fire location.
   - `url`: A thumbnail URL for visualizing the area around the wildfire event.

In [ ]:
import ee
import pandas as pd

# Authenticate and initialize Earth Engine
ee.Authenticate()
project_id = 'sdp-wildfire-detection-aze'
ee.Initialize(project=project_id)

# Function to mask clouds using the Sentinel-2 QA60 band
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load the CSV file (modify the path to your actual CSV file)
csv_path = '/content/drive/MyDrive/fire_nrt_J1V-C2_519849.csv'
df = pd.read_csv(csv_path)

image_urls =[]

# Assuming your CSV file has columns: 'LATITUDE', 'LONGITUDE', 'ACQ_DATE'
for index, row in df.iterrows():
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']

    # Convert acquisition date to 'YYYY-MM-DD' format
    start_date = pd.to_datetime(row['ACQ_DATE']).strftime('%Y-%m-%d')
    end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

    # Define the location of the fire event
    point = ee.Geometry.Point([longitude, latitude])
    # Define the bounds for the rectangular area
    buffer_size = 0.015  # Approx 1 km in degrees (modify as needed)
    coords = [
        [longitude - buffer_size, latitude - buffer_size],  # Bottom-left
        [longitude + buffer_size, latitude - buffer_size],  # Bottom-right
        [longitude + buffer_size, latitude + buffer_size],  # Top-right
        [longitude - buffer_size, latitude + buffer_size],  # Top-left
        [longitude - buffer_size, latitude - buffer_size]   # Closing the rectangle
    ]
    rectangle = ee.Geometry.Polygon(coords)

    # Load Sentinel-2 harmonized collection and apply cloud mask
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(rectangle)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                  .map(maskS2clouds)
                  .limit(40))

    # Check the size of the collection
    collection_size = collection.size().getInfo()
    print(f"Collection size for ({latitude}, {longitude}): {collection_size}")

    if collection_size > 0:
        # Select RGB bands (B4 = Red, B3 = Green, B2 = Blue)
        image = collection.median().select(['B4', 'B3', 'B2'])

        # Clip the image to the specific polygon (area of interest)
        clipped_image = image.clip(rectangle)

        # Generate a visualization URL for the clipped image
        url = clipped_image.getThumbURL({
            'min': 0, 'max': 0.5,  # Adjust the min/max for better visualization
            'dimensions': 512,
            'region': rectangle,
            'format': 'png'
        })
        image_urls.append({'latitude': latitude, 'longitude': longitude, 'url': url})  # Appending a dictionary


In [ ]:
image_urls

### Saving Image URLs to a CSV File (Optional)

This code converts the list of generated image URLs to a pandas DataFrame and saves it to a CSV file.


In [ ]:
# Convert the list of image URLs to a pandas DataFrame
image_urls_df = pd.DataFrame(image_urls)

# Save the DataFrame to a CSV file
output_csv_path = '/content/image_urls.csv'  # Modify this path as needed
image_urls_df.to_csv(output_csv_path, index=False)

print(f"Image URLs saved to {output_csv_path}")